In [1]:
!pip install -U trl accelerate peft -i https://pypi.org/simple/ bitsandbytes transformers trl huggingface_hub
%pip install -U datasets

Looking in indexes: https://pypi.org/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 15.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cubla

In [2]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging)
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from sklearn.model_selection import train_test_split

In [3]:
from google.colab import files


uploaded = files.upload()

Saving ft_data.xlsx to ft_data.xlsx


In [44]:
# filename = "/kaggle/input/sentiment-analysis-for-financial-news/all-data.csv"
filename="/content/ft_data.xlsx"

df = pd.read_excel(filename)
df.head()

,intent,query
0,Card: disable,"Need to block my card ASAP, think it's been co..."
1,Card: disable,"Freeze my card, pretty sure I left it at the b..."
2,Card: disable,Can you put a hold on my card? I can't find it...
3,Card: disable,"Ugh, lost my wallet. Disable my card before so..."
4,Card: disable,"Hey, I think my card's been nicked! Lock it do..."


In [48]:
X_train = list()
X_test = list()
for intent in df['intent'].unique():
    train, test  = train_test_split(df[df.intent==intent],
                                    train_size=0.4,test_size=0.3,
                                    random_state=42)
    X_train.append(train)
    X_test.append(test)

X_train[:1]

[           intent                                              query
 11  Card: disable               I want to block my card immediately.
 3   Card: disable  Ugh, lost my wallet. Disable my card before so...
 18  Card: disable  Can you assist me in pausing my card transacti...
 16  Card: disable   Put a hold on my card, I can't find it anywhere.
 13  Card: disable            How do I go about deactivating my card?
 2   Card: disable  Can you put a hold on my card? I can't find it...
 9   Card: disable  Something's fishy with my card. Shut it down u...
 19  Card: disable     Please deactivate my banking card temporarily.]

In [49]:
X_train = pd.concat(X_train).sample(frac=1, random_state=10)
X_test = pd.concat(X_test)
X_test

,intent,query
0,Card: disable,"Need to block my card ASAP, think it's been co..."
17,Card: disable,I need to block my card as I suspect it's been...
15,Card: disable,"I lost my wallet, can you disable my card to p..."
1,Card: disable,"Freeze my card, pretty sure I left it at the b..."
8,Card: disable,Put my card on ice; it's gone AWOL.
...,...,...
877,User Account: change password post login,What's the procedure to update my login passwo...
875,User Account: change password post login,Show me steps to change my user account password.
861,User Account: change password post login,"Need to update my current password, what's the..."
868,User Account: change password post login,Change password feature? Where's that at?


In [50]:
eval_idx = [idx for idx in df.index if idx not in list(train.index) + list(test.index)]
eval_idx[:5]
X_eval = df[df.index.isin(eval_idx)]
X_eval[:5]

,intent,query
0,Card: disable,"Need to block my card ASAP, think it's been co..."
1,Card: disable,"Freeze my card, pretty sure I left it at the b..."
2,Card: disable,Can you put a hold on my card? I can't find it...
3,Card: disable,"Ugh, lost my wallet. Disable my card before so..."
4,Card: disable,"Hey, I think my card's been nicked! Lock it do..."


In [51]:
X_eval = (X_eval
          .groupby('intent', group_keys=False)
          .apply(lambda x: x.sample(n=50, random_state=10, replace=True)))
X_train = X_train.reset_index(drop=True)

In [7]:
model_name = "microsoft/phi-2"

compute_dtype = getattr(torch, "float16")

# Model Configurations
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)

# Loading the Model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=bnb_config,
)

model.config.use_cache = False
model.config.pretraining_tp = 1

# Loading the Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          trust_remote_code=True,
                                         )
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [31]:
# ans=[]
# for query in X_test['query']:
#   prompt=f"""You have to serve as an Conversational Intent Classifier for the query given by the user,
#             below is the list from which you have to choose the intent.

#       List of Predefined Intents:
#       Card: disable
#       Card: enable
#       Card: get shipping status where is
#       Card: report stolen or lost
#       Global: get balance
#       Global: get routing number direct deposit info
#       Statement: get
#       Transaction: history
#       Transaction: report dispute incorrect
#       User Account: change email address
#       User Account: change mailing address
#       Spend Account: transfer funds
#       Spend Account: transfer funds checks
#       Spend Account: transfer funds external bank
#       Savings Account: get info view program
#       User Account: connect banks
#       Rewards: view offers
#       Rewards: opt in
#       Rewards: opt out
#       User Account: get help contact customer service',
#       Spending Tracker: get info',
#       Overdraft: opt out
#       User Account: get secure inbox messages
#       User Account: get fee plan info
#       Card: add new
#       Document: upload
#       Spend Account: find ATMs
#       Spend Account: get cash withdrawal and reload locations',
#       User Account: log out
#       Card: get info status
#       User Account: esign
#       Refer a Friend: get info
#       User Account: change phone number
#       User Account: edit profile name
#       Card: reset PIN
#       Card: cancel close
#       Card: temporary limit increase
#       Spend Account: consent to direct deposit',
#       User Account: closure request
#       Notifications: manage
#       Notifications: sign up for
#       Card: report not receiveda
#       Card: replace or upgrade
#       User Account: change password post login

#       Only return the intent and nothing else.
#       Return the intent for below query
#       {query} """
#   inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
#   generation_params: dict = {"max_length":500}
#   outputs = model.generate(**inputs,**generation_params)
#   completion = tokenizer.batch_decode(outputs)[0]

#   ans.append(completion)

In [52]:
def generate_prompt(data_point):
    return f"""The intent of the following query: '{data_point["query"]}' is {data_point["intent"]}""".strip()

def generate_test_prompt(data_point):
    return f"""The intent of the following query: '{data_point["query"]}' is""".strip()

In [53]:
X_train = pd.DataFrame(X_train.apply(generate_prompt, axis=1),
                       columns=["query"])

X_train.head()

,query
0,The intent of the following query: 'I'm planni...
1,The intent of the following query: 'Walk me th...
2,The intent of the following query: 'Tell me wh...
3,The intent of the following query: 'Where do I...
4,The intent of the following query: 'Just got a...


In [74]:
def predict(X_test, model, tokenizer):
    y_pred = []
    for i in tqdm(range(len(X_test))):
        prompt = X_test.iloc[i]["query"]
        pipe = pipeline(task="text-generation",
                        model=model,
                        tokenizer=tokenizer,
                        max_new_tokens = 50,
                        temperature = 0.0,
                       )
        result = pipe(prompt, pad_token_id=pipe.tokenizer.eos_token_id)
        answer = result[0]['generated_text'].lower()
        y_pred.append(answer)
    return y_pred

In [58]:
y_true = X_test.intent.lower()
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["query"])

train_data = Dataset.from_pandas(X_train)
eval_data = Dataset.from_pandas(X_eval)

In [59]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=[
    "q_proj",
    "up_proj",
    "o_proj",
    "k_proj",
    "down_proj",
    "gate_proj",
    "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
training_arguments = TrainingArguments(
    output_dir="logs",
    num_train_epochs=2,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8, # 4
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
    evaluation_strategy="epoch"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="query",
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
    max_seq_length=512,
)

trainer.train()

# Save trained model
trainer.model.save_pretrained("micrsofot_phi2_sentiment_sim-model")

Map:   0%|          | 0/352 [00:00<?, ? examples/s]

Map:   0%|          | 0/2200 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,3.361500,3.349810
2,1.525700,3.337296


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


NameError: name 'y_test' is not defined

In [75]:
y_pred = predict(X_test, model, tokenizer)

  0%|          | 0/264 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
100%|██████████| 264/264 [01:19<00:00,  3.32it/s]


In [78]:
for i in range(0,len(y_pred)):
  y_pred[i]= y_pred[i].split('is')[-1].lower()
  y_pred[i]= y_pred[i].strip()

In [1]:
y_pred

NameError: name 'y_pred' is not defined

In [73]:
y_true

0                                 Card: disable
17                                Card: disable
15                                Card: disable
1                                 Card: disable
8                                 Card: disable
                         ...                   
877    User Account: change password post login
875    User Account: change password post login
861    User Account: change password post login
868    User Account: change password post login
865    User Account: change password post login
Name: intent, Length: 264, dtype: object

In [80]:
cm = confusion_matrix(y_true, y_pred)

In [83]:
cm

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [85]:
cm_df = pd.DataFrame(cm,
                     index = y_true,
                     columns = y_true)

ValueError: Shape of passed values is (89, 89), indices imply (264, 264)